# Introduction

Fork from https://www.kaggle.com/augustmarvel/base-model-v2-user-level-solution

### This kernel is mainly made up of three parts:
* [**1. Data loading**](#Data loading)
* [**2. Data preprocessing**](#Data preprocessing)
* [**3. Model building**](#Model building)

###  Main of the kernel:
*  Chunk whole of data set  by the period as such a structure: 
* [210 days of training period, 45 days of gap period, 2 months of traget perod].  
* Aggregating data from the training period, ignoring the  gap period, getting the target from the traget period. 
* The valiation set is set to Dec-Jan which is the same monthly period  as the target peroid of the test set.

### Summary:
 In this competition, the data set is so unbalanced that it's hard to say whether our solution can beat all-zeros. After doing some basic EDA, there are some conclusions are for sure: 

1. if a customer will pay,  the  transaction will be highly likely happened at the first month, and no longer than two months after the customer shows up in first time. 
2. the minimum of transaction revenue is no less than 1E+07.
---
* For the first one, our test set has a 1.5 months' gap between the traget period  which means our taget is divided into two groups: the first  is the one who has already spent no less than 45 days on thinking whether to pay. The second is the  one who has payed for partial services and is going to pay for additional services. To the first group, the customers are terrific unlikely to pay. To the second one, the customers are likely to pay much the same as they payed before. Under those conditions, my prediction of the number of people to pay is 200 or so.
* For the second one, as we see, the prediction of our model is full of numbers less than 1E+07. But you'll get a worse score if you set those numbers to zero. Our model keeps betting wisely on minimize RMSE but the result keeps leaving away from the real numbers. 

### random thoughts:
* To set a user-defined objective function, which gives a high penalty once the floor level is breached, will be good for avoiding small values.
* Time features should be under the first priority.
* To the second group people, if it's possible to specify them by clustering.
* if the customers wil return after a full year of services are expired?
* the data set is lack of some important features such as page views of user's website. To the low volume users, why do they pay the bill for advance services if the free account already meets all the needs?

* Data are generated from this script : https://www.kaggle.com/qnkhuat/make-data-ready 
* Stacking part is from this script: https://www.kaggle.com/ashishpatel26/updated-bayesian-lgbm-xgb-cat-fe-kfold-cv

## Data loading

In [1]:
import numpy as np 
import pandas as pd 
import scipy
from datetime import datetime

import sys
import os
from os.path import join as pjoin

data_root = '../input/make-data-ready'
print(os.listdir(data_root))

pd.set_option('display.max_rows',200)

from sklearn.preprocessing import LabelEncoder

from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns


['train_5.pkl', 'train_1.pkl', 'train_2.pkl', 'train_7.pkl', 'test_2.pkl', 'test_1.pkl', 'train_6.pkl', 'train_8.pkl', 'train_0.pkl', 'train_4.pkl', '__output__.json', 'train_3.pkl', 'test_0.pkl']


In [2]:
def load_data(data='train',n=2):
    df = pd.DataFrame()
    for i in range(n) :
        if data=='train':
            if i > 8 :
                break
            dfpart = pd.read_pickle(pjoin(data_root,f'train_{i}.pkl'))
        elif data=='test':
            if i > 2 :
                break
            dfpart = pd.read_pickle(pjoin(data_root,f'test_{i}.pkl'))
        df = pd.concat([df,dfpart])
        del dfpart
    return df
        

In [3]:
df_train = load_data(n=9)
df_test = load_data('test',n=4)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

## Data preprocessing

In [4]:
df = pd.concat([df_train, df_test])

In [5]:
print(df.columns)

Index(['channelGrouping', 'date', 'fullVisitorId', 'visitId', 'visitNumber',
       'visitStartTime', 'device_browser', 'device_deviceCategory',
       'device_isMobile', 'device_operatingSystem', 'geoNetwork_city',
       'geoNetwork_continent', 'geoNetwork_country', 'geoNetwork_metro',
       'geoNetwork_networkDomain', 'geoNetwork_region',
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',
       'totals_newVisits', 'totals_pageviews', 'totals_sessionQualityDim',
       'totals_timeOnSite', 'totals_totalTransactionRevenue',
       'totals_transactionRevenue', 'totals_transactions', 'totals_visits',
       'trafficSource_adContent',
       'trafficSource_adwordsClickInfo.adNetworkType',
       'trafficSource_adwordsClickInfo.gclId',
       'trafficSource_adwordsClickInfo.isVideoAd',
       'trafficSource_adwordsClickInfo.page',
       'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign',
       'trafficSource_isTrueDirect', 'trafficSource_keyword',
       '

### Drop some features and items

In [6]:
# col_drop = ['Date_Year', 'Date_Month', 'Date_Week','Date_Hour','device_isMobile','device_deviceCategory',
#        'Date_Day', 'Date_Dayofweek', 'Date_Dayofyear', 'Date_Is_month_end',
#        'Date_Is_month_start', 'Date_Is_quarter_end', 'Date_Is_quarter_start',
#        'Date_Is_year_end', 'Date_Is_year_start','totals_visits',
#            'date','visitId','totals_totalTransactionRevenue','geoNetwork_city','geoNetwork_continent',
#             'geoNetwork_metro','geoNetwork_networkDomain',
# 'geoNetwork_region','geoNetwork_subContinent','trafficSource_adContent',
#             'trafficSource_adwordsClickInfo.adNetworkType','trafficSource_adwordsClickInfo.gclId',
# 'trafficSource_adwordsClickInfo.slot','trafficSource_campaign',
#             'trafficSource_keyword','trafficSource_referralPath','trafficSource_medium',
#             'customDimensions_value','customDimensions_index','trafficSource_source',
#            'totals_bounces','visitNumber','totals_newVisits']
col_drop = ['Date_Year', 'Date_Month', 'Date_Week','Date_Hour',
       'Date_Day', 'Date_Dayofweek', 'Date_Dayofyear', 'Date_Is_month_end',
       'Date_Is_month_start', 'Date_Is_quarter_end', 'Date_Is_quarter_start',
       'Date_Is_year_end', 'Date_Is_year_start','totals_visits',
            'customDimensions_value','customDimensions_index'
           ]
df.drop(col_drop, axis=1, inplace=True)

In [7]:
#replace empty fields with 'NA'
Nulls = ['(not set)', 'not available in demo dataset', '(not provided)', 
         'unknown.unknown', '/', 'Not Socially Engaged']
for null in Nulls:    
    df.replace(null, 'NA', inplace=True)

In [8]:
# country_drop=df.groupby('geoNetwork_country')['totals_transactions'].sum()[df.groupby('geoNetwork_country')['totals_transactions'].sum().sort_values()<4].index.tolist()
# df.loc[df[df.geoNetwork_country.isin(country_drop)].index,'geoNetwork_country'] = 'NaN'

# df.loc[df[~df.device_browser.isin(['Edge', 'Internet Explorer', 'Firefox', 'Safari', 'Chrome'])].index,'device_browser'] = 'NaN'
# df.loc[df[~df.device_operatingSystem.isin(['Android', 'iOS', 'Linux', 'Chrome OS', 'Windows', 'Macintosh'])].index,'device_operatingSystem'] = 'NaN'

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2109926 entries, 0 to 1588
Data columns (total 38 columns):
channelGrouping                                 object
date                                            datetime64[ns]
fullVisitorId                                   object
visitId                                         int64
visitNumber                                     int64
visitStartTime                                  int64
device_browser                                  object
device_deviceCategory                           object
device_isMobile                                 bool
device_operatingSystem                          object
geoNetwork_city                                 object
geoNetwork_continent                            object
geoNetwork_country                              object
geoNetwork_metro                                object
geoNetwork_networkDomain                        object
geoNetwork_region                               object
geoNetw

### Label encoding

In [10]:
# col = 'trafficSource_adwordsClickInfo.page'
# print(len(df[col].unique()))
# print(df[col].mode())
# print(df[col].describe)

In [11]:
col_lb = ['channelGrouping','device_browser', 'device_deviceCategory', 'device_operatingSystem', 
          'geoNetwork_city', 'geoNetwork_country', 'geoNetwork_metro', 'geoNetwork_networkDomain',
          'geoNetwork_region', 'geoNetwork_continent', 'geoNetwork_subContinent',
          'trafficSource_adContent', 'trafficSource_adwordsClickInfo.adNetworkType',
          'trafficSource_adwordsClickInfo.gclId', 'trafficSource_adwordsClickInfo.slot',
          'trafficSource_campaign', 'trafficSource_keyword', 'trafficSource_medium',
          'trafficSource_referralPath', 'trafficSource_source']
for col in col_lb:
    lb = LabelEncoder()
    df[col]=lb.fit_transform(df[col])

### Features to user level
There is also a feature called time_diff, which is directly coded in generating part. And this time- relative feature really works well

In [12]:
to_mode = ['channelGrouping', 'device_browser', 'device_deviceCategory', 
           'geoNetwork_networkDomain', 'geoNetwork_city', 'device_operatingSystem', 'geoNetwork_metro', 'geoNetwork_region',
          'geoNetwork_country', 'geoNetwork_continent', 
           'trafficSource_adwordsClickInfo.gclId', 
           'trafficSource_keyword', 'trafficSource_medium', 'trafficSource_referralPath', 'trafficSource_source',
          'totals_sessionQualityDim', 'trafficSource_isTrueDirect', 'totals_newVisits', 'device_isMobile',
          'trafficSource_adwordsClickInfo.isVideoAd', 'trafficSource_adwordsClickInfo.page']
to_sum = ['totals_timeOnSite', 'totals_pageviews', 'totals_hits', 'totals_totalTransactionRevenue', 'totals_transactions']
to_min = ['totals_timeOnSite', 'totals_pageviews', 'totals_hits', 'totals_totalTransactionRevenue', 'totals_transactions']
to_max = ['totals_timeOnSite', 'totals_pageviews', 'totals_hits', 'totals_totalTransactionRevenue', 'totals_transactions',
          'visitNumber']
to_mean = ['totals_timeOnSite', 'totals_pageviews', 'totals_hits', 'totals_totalTransactionRevenue', 'totals_transactions',
           'trafficSource_isTrueDirect', 'totals_newVisits', 'device_isMobile', 'trafficSource_adwordsClickInfo.isVideoAd',
          'trafficSource_adwordsClickInfo.page', 'totals_sessionQualityDim']
to_std = ['totals_timeOnSite', 'totals_pageviews', 'totals_hits', 'totals_totalTransactionRevenue', 'totals_transactions']
to_skew = ['totals_timeOnSite', 'totals_pageviews', 'totals_hits', 'totals_totalTransactionRevenue', 'totals_transactions']

to_time = 'visitStartTime'

In [13]:
# to_median = ['channelGrouping','device_browser','device_operatingSystem',
#              'geoNetwork_country','trafficSource_adwordsClickInfo.isVideoAd',
#              'trafficSource_isTrueDirect','trafficSource_adwordsClickInfo.page']
# to_sum =['totals_hits','totals_pageviews','totals_timeOnSite','totals_transactionRevenue', 'totals_transactions']
# to_mean =['totals_hits','totals_pageviews','totals_sessionQualityDim']
# to_std = ['totals_hits','totals_pageviews']
# to_time = 'visitStartTime'

### Time period
* the training set has a 46 days gap to its target set that is same as the test set 
* the training set has almost the same duration as the test set
* the valiation set is set to Dec-Jan which is the same monthly period  as the target peroid of the test set

In [14]:
# target_period = pd.date_range(start='2016-08-01',end='2018-12-01', freq='2MS')
# train_period = target_period.to_series().shift(periods=-210, freq='d',axis= 0)
# time_to = train_period[train_period.index>np.datetime64('2016-08-01')].astype('int')//10**9
# time_end = target_period.to_series().shift(periods=-45, freq='d',axis= 0)[4:]

In [15]:
target_period = pd.date_range(start='2016-08-01',end='2018-12-01', freq='2MS')
train_period = target_period.to_series().shift(periods=-8, freq='m').shift(periods=1, freq='d',axis= 0)
time_to = train_period[train_period.index>np.datetime64('2016-08-01')].astype('int')//10**9
time_end = target_period.to_series().shift(periods=-46, freq='d',axis= 0)[4:]

In [16]:
print(target_period)
print(train_period)
print(time_to)
print(time_end)

DatetimeIndex(['2016-08-01', '2016-10-01', '2016-12-01', '2017-02-01',
               '2017-04-01', '2017-06-01', '2017-08-01', '2017-10-01',
               '2017-12-01', '2018-02-01', '2018-04-01', '2018-06-01',
               '2018-08-01', '2018-10-01', '2018-12-01'],
              dtype='datetime64[ns]', freq='2MS')
2016-01-01   2016-08-01
2016-03-01   2016-10-01
2016-05-01   2016-12-01
2016-07-01   2017-02-01
2016-09-01   2017-04-01
2016-11-01   2017-06-01
2017-01-01   2017-08-01
2017-03-01   2017-10-01
2017-05-01   2017-12-01
2017-07-01   2018-02-01
2017-09-01   2018-04-01
2017-11-01   2018-06-01
2018-01-01   2018-08-01
2018-03-01   2018-10-01
2018-05-01   2018-12-01
Freq: 2MS, dtype: datetime64[ns]
2016-09-01    1491004800
2016-11-01    1496275200
2017-01-01    1501545600
2017-03-01    1506816000
2017-05-01    1512086400
2017-07-01    1517443200
2017-09-01    1522540800
2017-11-01    1527811200
2018-01-01    1533081600
2018-03-01    1538352000
2018-05-01    1543622400
Freq: 2MS, 

### Test data

In [17]:
# def cal_mode(x):
#     return max(map(lambda val: (x.count(val), val), set(x)))[1]

In [18]:
# helper function to calculate mode, faster than pandas built in mode function
from collections import defaultdict
def cal_mode(a):
    d = defaultdict(int)
    for i in a:
        d[i] += 1
    return sorted(d.items(), key=lambda x: x[1], reverse=True)[0][0]

# lVals = [1,2,3,1,2,1,1,1,3,2,2,1]
# print (cal_mode(lVals))         

In [19]:
df_test.shape

(401589, 54)

In [20]:
%%time
user_x = df.iloc[df_train.shape[0]:,:]
i = len(time_to) - 1 
# user_x = df[(df.visitStartTime>=(time_to.index.astype('int')//10**9)[i]) & (df.visitStartTime<(time_end.index.astype('int')//10**9)[i])]
test_x = user_x.groupby('fullVisitorId').agg(
    channelGrouping = ('channelGrouping', lambda x: cal_mode(x)),
    device_browser = ('device_browser', lambda x: cal_mode(x)),
    deviceCategory = ('device_deviceCategory', lambda x: cal_mode(x)), 
    operatingSystem = ('device_operatingSystem', lambda x: cal_mode(x)), 
    networkDomain = ('geoNetwork_networkDomain', lambda x: cal_mode(x)), 
    city = ('geoNetwork_city', lambda x: cal_mode(x)),
    metro = ('geoNetwork_metro', lambda x: cal_mode(x)),
    region = ('geoNetwork_region', lambda x: cal_mode(x)),
    country = ('geoNetwork_country', lambda x: cal_mode(x)),
    continent = ('geoNetwork_continent', lambda x: cal_mode(x)),
    adwordsClickInfo_gclId = ('trafficSource_adwordsClickInfo.gclId', lambda x: cal_mode(x)),
    keyword = ('trafficSource_keyword', lambda x: cal_mode(x)),
    medium = ('trafficSource_medium', lambda x: cal_mode(x)),
    referralPath = ('trafficSource_referralPath', lambda x: cal_mode(x)),
    source = ('trafficSource_source', lambda x: cal_mode(x)),
    isTrueDirect = ('trafficSource_isTrueDirect', lambda x: cal_mode(x)),
    isVideoAd = ('trafficSource_adwordsClickInfo.isVideoAd', lambda x: cal_mode(x)),
    adwordsClickInfo_page = ('trafficSource_adwordsClickInfo.page', lambda x: cal_mode(x)),
    totals_sessionQualityDim = ('totals_sessionQualityDim', lambda x: cal_mode(x)),
    newVisits = ('totals_newVisits', lambda x: cal_mode(x)),
    isMobile = ('device_isMobile', lambda x: cal_mode(x))
)

test_x = pd.concat([test_x,
                     user_x.groupby('fullVisitorId')['visitStartTime'].agg(['min','max']).add_suffix('_time').sub(time_to.values[i]).abs(),
                     user_x.groupby('fullVisitorId')['visitStartTime'].apply(lambda x: x.max() -x.min()).rename('time_diff'),
                     user_x.groupby('fullVisitorId')[to_sum].sum().add_suffix('_sum'),
                     user_x.groupby('fullVisitorId')[to_mean].mean().add_suffix('_mean'),
                     user_x.groupby('fullVisitorId')[to_min].min().add_suffix('_min'),
                     user_x.groupby('fullVisitorId')[to_max].max().add_suffix('_max'),
                     # user_x.groupby('fullVisitorId')[to_skew].skew().add_suffix('_skew'),
                     user_x.groupby('fullVisitorId')[to_std].std(ddof=0).add_suffix('_std')], axis=1).reset_index()

test_x['month'] = str(time_end.dt.month[i])
test_x['year'] = str(time_end.dt.year[i])

test_x.to_pickle('test_clean')

CPU times: user 4min 49s, sys: 4.32 s, total: 4min 53s
Wall time: 4min 50s


### Training data

In [21]:
%%time
blocks = 9

for i in range(blocks):
    print(i, time_to[i])
    user_x = df[(df.visitStartTime>=(time_to.index.astype('int')//10**9)[i]) & (df.visitStartTime<(time_end.index.astype('int')//10**9)[i])]
    user_y = df[(df.visitStartTime>=time_end.values[i].astype('int')//10**9) & (df.visitStartTime<time_end.values[i+1].astype('int')//10**9)]
    
    train_x = user_x.groupby('fullVisitorId').agg(
        channelGrouping = ('channelGrouping', lambda x: cal_mode(x)),
        device_browser = ('device_browser', lambda x: cal_mode(x)),
        deviceCategory = ('device_deviceCategory', lambda x: cal_mode(x)), 
        operatingSystem = ('device_operatingSystem', lambda x: cal_mode(x)), 
        networkDomain = ('geoNetwork_networkDomain', lambda x: cal_mode(x)), 
        city = ('geoNetwork_city', lambda x: cal_mode(x)),
        metro = ('geoNetwork_metro', lambda x: cal_mode(x)),
        region = ('geoNetwork_region', lambda x: cal_mode(x)),
        country = ('geoNetwork_country', lambda x: cal_mode(x)),
        continent = ('geoNetwork_continent', lambda x: cal_mode(x)),
        adwordsClickInfo_gclId = ('trafficSource_adwordsClickInfo.gclId', lambda x: cal_mode(x)),
        keyword = ('trafficSource_keyword', lambda x: cal_mode(x)),
        medium = ('trafficSource_medium', lambda x: cal_mode(x)),
        referralPath = ('trafficSource_referralPath', lambda x: cal_mode(x)),
        source = ('trafficSource_source', lambda x: cal_mode(x)),
        isTrueDirect = ('trafficSource_isTrueDirect', lambda x: cal_mode(x)),
        isVideoAd = ('trafficSource_adwordsClickInfo.isVideoAd', lambda x: cal_mode(x)),
        adwordsClickInfo_page = ('trafficSource_adwordsClickInfo.page', lambda x: cal_mode(x)),
        totals_sessionQualityDim = ('totals_sessionQualityDim', lambda x: cal_mode(x)),
        newVisits = ('totals_newVisits', lambda x: cal_mode(x)),
        isMobile = ('device_isMobile', lambda x: cal_mode(x))
    )
    
    train_x = pd.concat([train_x,
                     user_x.groupby('fullVisitorId')['visitStartTime'].agg(['min','max']).add_suffix('_time').sub(time_to.values[i]).abs(),
                     user_x.groupby('fullVisitorId')['visitStartTime'].apply(lambda x: x.max() -x.min()).rename('time_diff'),
                     user_x.groupby('fullVisitorId')[to_sum].sum().add_suffix('_sum'),
                     user_x.groupby('fullVisitorId')[to_mean].mean().add_suffix('_mean'),
                     user_x.groupby('fullVisitorId')[to_min].min().add_suffix('_min'),
                     user_x.groupby('fullVisitorId')[to_max].max().add_suffix('_max'),
                     # user_x.groupby('fullVisitorId')[to_skew].skew().add_suffix('_skew'),
                     user_x.groupby('fullVisitorId')[to_std].std(ddof=0).add_suffix('_std')], axis=1).reset_index()
    
    train_x['month'] = str(time_end.dt.month[i])
    train_x['year'] = str(time_end.dt.year[i])
    
    merged = train_x.merge(user_y.groupby('fullVisitorId')['totals_transactionRevenue'].sum().reset_index(),\
                          how='left', on='fullVisitorId')
    
    user_ret = set(user_y['fullVisitorId'])
    merged['ret'] = merged['fullVisitorId'].isin(user_ret)
    
    merged.to_pickle('train_clean' + str(i))

0 1491004800
1 1496275200
2 1501545600
3 1506816000
4 1512086400
5 1517443200
6 1522540800
7 1527811200
8 1533081600
CPU times: user 50min 49s, sys: 46.4 s, total: 51min 35s
Wall time: 51min 2s


In [23]:
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                      df_train:  2.6 GiB
                            df: 722.1 MiB
                       df_test: 622.1 MiB
                        merged: 221.5 MiB
                       train_x: 215.6 MiB
                        test_x: 176.1 MiB
                        user_x: 171.0 MiB
                        user_y: 46.0 MiB
                      user_ret:  4.0 MiB
                           _ii:  3.2 KiB
